In [4]:
import numpy as np
import torch

In [34]:
L1_weights = torch.load('L1_Weights.pt', weights_only=True).int().numpy()
L1_bias = torch.load('L1_Bias.pt', weights_only=True).int().numpy()

L2_weights = torch.load('L2_Weights.pt', weights_only=True).int().numpy()
L2_bias = torch.load('L2_Bias.pt', weights_only=True).int().numpy()

img = torch.load('Image_input.pt', weights_only=True).int().numpy()
print(img.shape)

(100, 16)


In [89]:
y1 = img[54] @ L1_weights + L1_bias
print(y1)

[2 2 2 2 0 2 0 0]


In [90]:
y2 = np.array([1, 1, 1, 1, 0, 1, 0, 0]) @ L2_weights + L2_bias 

print(y2)

[4 2 5 3]


In [62]:
print(L1_weights)

[[0 0 0 0 0 0 0 0]
 [0 0 0 1 0 1 1 1]
 [0 0 0 0 0 0 1 0]
 [0 0 0 0 0 1 0 0]
 [1 0 0 1 0 0 0 0]
 [0 0 0 0 0 1 0 1]
 [0 0 0 0 0 0 0 0]
 [0 1 1 0 0 0 0 0]
 [0 1 1 0 0 0 0 0]
 [0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0]
 [1 0 0 1 0 0 0 0]
 [1 0 0 0 1 1 0 1]
 [0 0 0 0 0 0 0 0]
 [0 0 0 0 0 1 0 0]
 [1 0 0 0 1 1 0 1]]


In [63]:
print(L1_bias)

[0 0 0 0 0 0 0 0]


In [68]:
w1_prog = np.vstack([L1_weights, L1_bias.reshape(1,-1)])

print(w1_prog)

[[0 0 0 0 0 0 0 0]
 [0 0 0 1 0 1 1 1]
 [0 0 0 0 0 0 1 0]
 [0 0 0 0 0 1 0 0]
 [1 0 0 1 0 0 0 0]
 [0 0 0 0 0 1 0 1]
 [0 0 0 0 0 0 0 0]
 [0 1 1 0 0 0 0 0]
 [0 1 1 0 0 0 0 0]
 [0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0]
 [1 0 0 1 0 0 0 0]
 [1 0 0 0 1 1 0 1]
 [0 0 0 0 0 0 0 0]
 [0 0 0 0 0 1 0 0]
 [1 0 0 0 1 1 0 1]
 [0 0 0 0 0 0 0 0]]


In [66]:
print(L2_weights)

[[1 0 1 1]
 [1 0 1 0]
 [1 0 1 0]
 [1 0 1 0]
 [1 0 0 1]
 [0 1 0 1]
 [1 1 0 0]
 [0 1 0 1]]


In [67]:
print(L2_bias)

[0 1 1 1]


In [69]:
w2_prog = np.vstack([L2_weights, L2_bias.reshape(1,-1)])

print(w2_prog)

[[1 0 1 1]
 [1 0 1 0]
 [1 0 1 0]
 [1 0 1 0]
 [1 0 0 1]
 [0 1 0 1]
 [1 1 0 0]
 [0 1 0 1]
 [0 1 1 1]]


In [14]:
def generate_pwl(binary_string, rise_fall_time='1us', time_step='100us', high_voltage = 3, low_voltage = 0, read_high = 0.5, read_low = 0):
    """
    Generate a PWL signal from a binary string with rise and fall times.

    Parameters:
    - binary_string (str): Binary string where '1' represents 1.5V and '0' represents 0V.
    - rise_fall_time (str): Rise and fall time in microseconds (default '0.1us').
    - time_step (str): Time step in microseconds (default '100us').

    Returns:
    - pwl_signal (str): A string representing the PWL signal.
    """

    # Convert 'rise_fall_time' and 'time_step' to float (in microseconds)
    rise_fall_time = int(rise_fall_time.replace('us', ''))
    time_step = int(time_step.replace('us', ''))

    # Convert the binary string to a list of voltages: 1 -> 1.5V, 0 -> 0V
    voltage_map_write = {'1': high_voltage, '0': low_voltage}
    voltage_map_read = {'1': read_high, '0': read_low}
    # Initialize PWL signal
    pwl_signal = f"pwl(time, 0us, {voltage_map_write[binary_string[0]]}V"

    # Current time in microseconds (start at 0)
    current_time = 0
    read_mode = False
    # Iterate through the binary string and generate the PWL signal
    for i in range(len(binary_string) - 1):
        if binary_string[i] == 'x':
            read_mode = True
            continue
        # print(binary_string[i])
        if read_mode:
            current_voltage = voltage_map_read[binary_string[i]]
            next_voltage = voltage_map_read[binary_string[i + 1]]
        else:
            if  binary_string[i+1] == 'x':
              current_voltage = voltage_map_write[binary_string[i]]
              next_voltage = voltage_map_read[binary_string[i + 2]]
            else:
              current_voltage = voltage_map_write[binary_string[i]]
              next_voltage = voltage_map_write[binary_string[i + 1]]

        # Add the current point
        pwl_signal += f", {current_time+time_step}us, {current_voltage}V"

        # Transition time for rise or fall (0.1 us)
        if current_voltage != next_voltage:
            # Rise or fall from the current voltage to the next voltage
            transition_time = current_time + rise_fall_time  # Midway for transition
            pwl_signal += f", {transition_time + time_step}us, {next_voltage}V"

        # Update time
        current_time += time_step

    # Add the last point (voltage for the last character in the binary string)
    if read_mode:
        final_voltage = voltage_map_read[binary_string[-1]]
    else:
        final_voltage = voltage_map_write[binary_string[-1]]
    pwl_signal += f", {current_time + time_step}us, {final_voltage}V"

    # Close the PWL signal string
    pwl_signal += ")"

    return pwl_signal

# Example usage:
binary_string = "101x10"
pwl_signal = generate_pwl(binary_string)

# Output the generated PWL signal
print(pwl_signal)


pwl(time, 0us, 3V, 100us, 3V, 101us, 0V, 200us, 0V, 201us, 3V, 300us, 3V, 301us, 0.5V, 400us, 0.5V, 401us, 0V, 500us, 0V)


In [31]:
def pwl_array(weights, image):
  pwl = {}
  # print(image.shape)
  if len(image.shape) == 3:
    flattened_image = image.reshape(image.shape[0], -1)
  elif len(image.shape) == 2:
    flattened_image = np.array([image.reshape(-1)])
  # print(flattened_image.shape)
  for i in range(16):
    bin_str = "0"*i + "1" + "0"*(15-i)
    # print(bin_str)
    bin_str += "x"
    for j in range(flattened_image.shape[0]):
      bin_str += str(flattened_image[j][i])
    # print(bin_str)
    pwl[f"WL_{i}"] = generate_pwl(bin_str, rise_fall_time='1us', time_step='100us', high_voltage = 3, low_voltage = 1)


  for i in range(8):
    bin_str = ""
    for j in range(16):
      bin_str += '1' if weights[i][j] > 0.5 else '0'       #ENTER CONDITION FOR WEIGHTS BINARISATION
    bin_str += "x"
    for j in range(flattened_image.shape[0]):
      bin_str += "0"
    pwl[f"BL_{i}"] = generate_pwl(bin_str, rise_fall_time='1us', time_step='100us', high_voltage = 0, low_voltage = 2, read_low = 0)

  return pwl

In [50]:
print(L1_weights)

[[0 0 0 0 0 0 0 0]
 [0 0 0 1 0 1 1 1]
 [0 0 0 0 0 0 1 0]
 [0 0 0 0 0 1 0 0]
 [1 0 0 1 0 0 0 0]
 [0 0 0 0 0 1 0 1]
 [0 0 0 0 0 0 0 0]
 [0 1 1 0 0 0 0 0]
 [0 1 1 0 0 0 0 0]
 [0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0]
 [1 0 0 1 0 0 0 0]
 [1 0 0 0 1 1 0 1]
 [0 0 0 0 0 0 0 0]
 [0 0 0 0 0 1 0 0]
 [1 0 0 0 1 1 0 1]]


In [36]:
pwl_array(L1_weights.T, img[0].reshape(4,4))

{'WL_0': 'pwl(time, 0us, 3V, 100us, 3V, 101us, 1V, 200us, 1V, 300us, 1V, 400us, 1V, 500us, 1V, 600us, 1V, 700us, 1V, 800us, 1V, 900us, 1V, 1000us, 1V, 1100us, 1V, 1200us, 1V, 1300us, 1V, 1400us, 1V, 1500us, 1V, 1600us, 1V, 1601us, 0V, 1700us, 0V)',
 'WL_1': 'pwl(time, 0us, 1V, 100us, 1V, 101us, 3V, 200us, 3V, 201us, 1V, 300us, 1V, 400us, 1V, 500us, 1V, 600us, 1V, 700us, 1V, 800us, 1V, 900us, 1V, 1000us, 1V, 1100us, 1V, 1200us, 1V, 1300us, 1V, 1400us, 1V, 1500us, 1V, 1600us, 1V, 1601us, 0.5V, 1700us, 0.5V)',
 'WL_2': 'pwl(time, 0us, 1V, 100us, 1V, 200us, 1V, 201us, 3V, 300us, 3V, 301us, 1V, 400us, 1V, 500us, 1V, 600us, 1V, 700us, 1V, 800us, 1V, 900us, 1V, 1000us, 1V, 1100us, 1V, 1200us, 1V, 1300us, 1V, 1400us, 1V, 1500us, 1V, 1600us, 1V, 1601us, 0.5V, 1700us, 0.5V)',
 'WL_3': 'pwl(time, 0us, 1V, 100us, 1V, 200us, 1V, 300us, 1V, 301us, 3V, 400us, 3V, 401us, 1V, 500us, 1V, 600us, 1V, 700us, 1V, 800us, 1V, 900us, 1V, 1000us, 1V, 1100us, 1V, 1200us, 1V, 1300us, 1V, 1400us, 1V, 1500us, 1V, 1

In [70]:
import torch
from torch.utils.data import DataLoader, TensorDataset
import torch.nn as nn
import torch.optim as optim
import matplotlib.pyplot as plt

In [71]:
# Load the saved datasets
X_train, y_train = torch.load("training_data_500.pt")
X_val, y_val = torch.load("validation_data_60.pt")
X_test, y_test = torch.load("testing_data_100.pt")

# Create DataLoader for batching
train_dataset = TensorDataset(X_train, y_train)
val_dataset = TensorDataset(X_val, y_val)
test_dataset = TensorDataset(X_test, y_test)

train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=32, shuffle=False)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)

# Define the neural network model
class SimpleNN(nn.Module):
    def __init__(self):
        super(SimpleNN, self).__init__()
        self.fc1 = nn.Linear(16, 8)  # 16 input neurons, 8 hidden neurons
        self.fc2 = nn.Linear(8, 4)   # 8 hidden neurons, 4 output neurons (for 4 classes)
        self.activation = nn.Tanh()  # Activation function

    def forward(self, x):
        x = self.activation(self.fc1(x))
        x = torch.clamp(x, min=0.0, max=1.0)  # Clamp outputs between 0 and 1
        x = self.fc2(x)
        return x

C:\Users\USER\AppData\Local\Temp\ipykernel_10160\920616709.py:2: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  X_train, y_train = torch.load("training_data_500.pt")
C:\Users

In [72]:
model = SimpleNN()

# Load the state dict into the model
model.load_state_dict(torch.load("new_model.pth", weights_only=True))

<All keys matched successfully>

In [73]:
def test_model(model, test_loader):
    model.eval()
    correct = 0
    total = 0

    with torch.no_grad():
        for inputs, labels in test_loader:
            outputs = model(inputs)
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

    accuracy = 100 * correct / total
    print(f"Test Accuracy: {accuracy:.2f}%")

# Test the trained model
test_model(model, test_loader)

Test Accuracy: 71.00%
